In [ ]:
!pip install transformers datasets tokenizers huggingface -q
!pip install datasets evaluate transformers[sentencepiece] -q
!apt install git-lfs -q
!pip install py7zr -q

In [ ]:
import pandas as pd
import os
import torch
from transformers import AutoModelForMaskedLM,LongformerModel
from transformers import AutoTokenizer
from transformers import DataCollatorForLanguageModeling
from tqdm import tqdm
from datasets import Dataset
import warnings
from sklearn.model_selection import train_test_split
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset,DatasetDict
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset

In [ ]:
# import pandas as pd

# df_train = pd.read_csv("train.csv")
# df_test = pd.read_csv("test.csv")

In [ ]:
# for i in range(len(df_train)):
#     transcript = str(df_train['transcript'][i]).replace('#','').replace('\n',' ')
#     df_train['transcript'][i] = transcript

In [ ]:
# for i in range(len(df_test)):
#     transcript = str(df_train['transcript'][i]).replace('#','').replace('\n',' ')
#     df_train['transcript'][i] = transcript

In [ ]:
# df_test.to_csv("test2.csv")

In [ ]:
from transformers import AutoTokenizer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

max_input_length = 8192
max_output_length = 512

In [ ]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["src"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["tgt"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [ ]:
data = pd.read_json(r"/kaggle/input/dataset/elitr_minuting_dataset1.txt", lines=True)
df_train , df_val = train_test_split(data,test_size =0.2) 
df_train = Dataset.from_pandas(df_train)
df_val = Dataset.from_pandas(df_val)
df = DatasetDict()
df['train'] = df_train
df['val'] = df_val

In [ ]:
train_dataset = df_train
val_dataset = df_val

In [ ]:
df_val

In [ ]:
batch_size = 10

In [ ]:
train_dataset = train_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns = ['src', 'tgt', '__index_level_0__']
)

In [ ]:
val_dataset = val_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=['src', 'tgt', '__index_level_0__']
)

In [ ]:
train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)
val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

In [ ]:
!pip install transformers --upgrade


In [ ]:
from transformers import AutoModelForSeq2SeqLM,LongformerModel

In [ ]:
led = LongformerModel.from_pretrained("allenai/led-base-16384", gradient_checkpointing=True, use_cache=False).to(device)

In [ ]:
# set generate hyperparameters
led.config.num_beams = 2
led.config.max_length = 512
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
!pip install rouge_score

In [ ]:
rouge = load_metric("rouge")

In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=10,
    per_device_eval_batch_size=5,
    fp16=True,
    output_dir="./",
    logging_steps=5,
    eval_steps=10,
    save_steps=10,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
)

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
!nvidia-smi

In [ ]:
trainer = Seq2SeqTrainer(
    model=led,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('/dialogsum/')

In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("knkarthick/MEETING-SUMMARY-BART-LARGE-XSUM-SAMSUM-DIALOGSUM-AMI")
model = AutoModelForSeq2SeqLM.from_pretrained("knkarthick/MEETING-SUMMARY-BART-LARGE-XSUM-SAMSUM-DIALOGSUM-AMI")

# Load custom dataset from CSV file
data =pd.read_json(r"/kaggle/input/dataset/elitr_minuting_dataset1.txt", lines=True)

# Tokenize input and output texts
input_ids = tokenizer.batch_encode_plus(data["src"].tolist(), padding=True, truncation=True, return_tensors="pt")["input_ids"]
output_ids = tokenizer.batch_encode_plus(data["tgt"].tolist(), padding=True, truncation=True, return_tensors="pt")["input_ids"]

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    num_train_epochs=100,
    logging_steps=1000,
    save_steps=5000,
    eval_steps=5000,
    warmup_steps=500,
    weight_decay=0.01,
    learning_rate=2e-5,
    logging_dir="./logs",
    overwrite_output_dir=True
)

# Set up trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=list(zip(input_ids, output_ids)),
    eval_dataset=list(zip(input_ids, output_ids)),
    data_collator=lambda data: {'input_ids': torch.stack([item[0] for item in data]),
                                'attention_mask': torch.stack([item[1] for item in data]),
                                'decoder_input_ids': torch.stack([item[0] for item in data]),
                                'decoder_attention_mask': torch.stack([item[1] for item in data]),
                                'labels': torch.stack([item[0] for item in data])}
)

# Train the model
trainer.train()

# Generate summaries for test inputs
test_input_texts = ["Test input 1", "Test input 2", "Test input 3"]
test_input_ids = tokenizer.batch_encode_plus(test_input_texts, padding=True, truncation=True, return_tensors="pt")["input_ids"]
test_output = model.generate(test_input_ids)
test_summaries = tokenizer.batch_decode(test_output, skip_special_tokens=True)


/usr/local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[percpu.cc : 552] RAW: rseq syscall failed with errno 1
[percpu.cc : 524] RAW: Failed to initialize RSEQ VCPUs in mode 1, error = 1
[percpu.cc : 557] RAW: rseq syscall failed with errno 1
2023-03-14 09:19:53.410313: E tensorflow/core/framework/op_kernel.cc:1676] OpKernel ('op: "TPURoundRobin" device_type: "CPU"') for unknown op: TPURoundRobin
2023-03-14 09:19:53.410392: E tensorflow/core/framework/op_kernel.cc:1676] OpKernel ('op: "TpuHandleToProtoKey" device_type: "CPU"') for unknown op: TpuHandleToProtoKey


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/usr/local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 103
  Num Epochs = 100
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 2600
  Number of trainable parameters = 509231104
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: kwagh20ite. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss


KeyboardInterrupt: 